In [1]:
# na primeira rodada, tirar o hashtag antes do ponto de exclamação e rodar essa linha
#!pip install netCDF4 numpy plotly ipywidgets matplotlib

In [2]:
# montando o drive no colab (vai abrir uma janela pedindo autorização para acessar o drive)
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
# Importar bibliotecas
import netCDF4 as nc
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
# Função para encontrar o índice mais próximo
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# Função para obter a precipitação em um ponto específico
def get_precipitation_at_point(nc_file, lat, lon, buffer=0.5):
    dataset = nc.Dataset(nc_file)
    lats = dataset.variables['lat'][:]
    lons = dataset.variables['lon'][:]
    precipitation = dataset.variables['precipitation'][:]  # Supondo que a variável se chame 'precipitation'

    lat_idx = find_nearest(lats, lat)
    lon_idx = find_nearest(lons, lon)

    precip_value = precipitation[0, lat_idx, lon_idx]

    # Determinar os índices para o buffer
    lat_min_idx = find_nearest(lats, lat - buffer)
    lat_max_idx = find_nearest(lats, lat + buffer)
    lon_min_idx = find_nearest(lons, lon - buffer)
    lon_max_idx = find_nearest(lons, lon + buffer)

    # Verificar e corrigir a ordem dos índices de latitude
    if lat_min_idx > lat_max_idx:
        lat_min_idx, lat_max_idx = lat_max_idx, lat_min_idx

    # Verificar e corrigir a ordem dos índices de longitude
    if lon_min_idx > lon_max_idx:
        lon_min_idx, lon_max_idx = lon_max_idx, lon_min_idx

    # Extrair a área de interesse
    precip_area = precipitation[0,lat_min_idx:lat_max_idx+1, lon_min_idx:lon_max_idx+1]
    lat_area = lats[lat_min_idx:lat_max_idx+1]
    lon_area = lons[lon_min_idx:lon_max_idx+1]

    return precip_value, precip_area, lat_area, lon_area

# Função para calcular o dia juliano
def dia_juliano(data):
    """
    Calcula o dia juliano para uma data fornecida.
    :param data: uma instância de datetime.date
    :return: o dia juliano (int)
    """
    # Calcula a diferença entre a data fornecida e o primeiro dia do ano
    primeiro_dia_do_ano = datetime.date(data.year, 1, 1)
    delta = data - primeiro_dia_do_ano

    # O dia juliano é o delta mais um
    return delta.days + 1

In [5]:
# Criar widgets para entrada de coordenadas
lat_input = widgets.FloatText(description='Latitude:')
lon_input = widgets.FloatText(description='Longitude:')
day_input = widgets.IntText(description='Dia:', min=1, max=31)
month_input = widgets.IntText(description='Mês:', min=1, max=12)
year_input = widgets.IntText(description='Ano:', min=1900, max=2030)
button = widgets.Button(description='Get Precipitation')

In [6]:
# Função para exibir o resultado
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        lat = lat_input.value
        lon = lon_input.value
        day = day_input.value
        month = month_input.value
        year = year_input.value

        try:
            date = datetime.date(year, month, day)
        except ValueError:
            print("Data inválida. Por favor, insira um dia, mês e ano válidos.")
            return

        # Formatar a data para o formato AAAAMMDD
        data_entrada = datetime.date(year, month, day)
        dia_juliano_saida = dia_juliano(data_entrada)
        nc_file = f"{year}{dia_juliano_saida}.nc"
        caminho_arquivo = f'/content/drive/MyDrive/MSWEP_V280/NRT/Daily/{nc_file}'

        try:
            precip, precip_area, lat_area, lon_area = get_precipitation_at_point(caminho_arquivo, lat, lon)
            plt.figure(figsize=(8, 6))
            plt.contourf(lon_area, lat_area, precip_area, cmap='Blues')
            plt.colorbar(label='Precipitação (mm)')
            plt.scatter(lon, lat, color='red', label='Ponto de Interesse')
            plt.legend()
            plt.title(f'Precipitação em {data_entrada} ao redor de ({lat}, {lon})')
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')
            plt.show()
            precip_media_area = precip_area[int(precip_area.shape[0] / 2), int(precip_area.shape[1] / 2)]

            print(f'Precipitação em ({lat}, {lon}) na data {date}: {precip} mm e a precipitacao média na área plotada é: {precip_media_area} mm')

        except FileNotFoundError as e:
            print(e)

button.on_click(on_button_click)

# Exibir widgets
display(lat_input, lon_input, day_input, month_input, year_input, button, output)

FloatText(value=0.0, description='Latitude:')

FloatText(value=0.0, description='Longitude:')

IntText(value=0, description='Dia:')

IntText(value=0, description='Mês:')

IntText(value=0, description='Ano:')

Button(description='Get Precipitation', style=ButtonStyle())

Output()